#Week 3 Lab: Convolutional Neural Networks

In this example, we will use Keras to develop convolutional neural network models.

Here's an example of transfer learning for image classification. We replace the last layer of inceptionV3 with a custom classification layer, which allows user-defined categories not present in the ImageNet challenge.
In this example, we are going to use ImageNet models to classify dogs and cats (https://www.kaggle.com/c/dogs-vs-cats). A subset of the images could be found at https://www.dropbox.com/sh/sun8qns27gb9hkc/AAAySZPWAioQM6GQIor-xKtZa?dl=0.

Tailored from examples in the Keras documentation: https://keras.io/



Kun-Hsing Yu

Updated and simplified on April 3, 2023

In [ ]:
# import tensorflow backend and keras
import tensorflow as tf

# using inception_V3 in this example, also works on other pre-trained models 
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K


### Download the example images

In [ ]:
!wget https://www.dropbox.com/s/v4vxu296ad9h69y/train.tar
!wget https://www.dropbox.com/s/hp25txiaxbdfvv0/validation.tar
!tar -xf train.tar
!tar -xf validation.tar

--2023-04-03 23:30:38--  https://www.dropbox.com/s/v4vxu296ad9h69y/train.tar
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/v4vxu296ad9h69y/train.tar [following]
--2023-04-03 23:30:39--  https://www.dropbox.com/s/raw/v4vxu296ad9h69y/train.tar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc96859d792f8c5a22e7a4fccd26.dl.dropboxusercontent.com/cd/0/inline/B5gP778yqwzfB1zIvZt3r6xAxfatMxpgMRb3fBM7ZEfkPtjrUUIYVUneGPwmbzrFs_GGQmGm-VRrsFMBs-hzo0y6K5mt3kLVhc8o9hUjS6IOY6-KwoWzMUQRHz0wsk8gyNc4WDjKsfpWM4W_NEV6_jD7_tT0RLVTZKGP7y42mUuI2g/file# [following]
--2023-04-03 23:30:40--  https://uc96859d792f8c5a22e7a4fccd26.dl.dropboxusercontent.com/cd/0/inline/B5gP778yqwzfB1zIvZt3r6xAxfatMxpgMRb3fBM7ZEfkPtjrUUIYVUneGPwmbzrFs_GGQmGm-VRrsFMBs-hzo0

## Quiz Question 1
What's in the dataset?

In [ ]:
# Your answer here
############################################################
!ls 
!echo "##### In train/ #####"
!ls train
!echo "##### In validation/ #####"
!ls validation
!echo "##### In train/cats/ #####"
!ls train/cats/ |head
!echo "##### In train/dogs/ #####"
!ls train/dogs/ |head
!echo "##### In validation/cats/ #####"
!ls validation/cats/ |head
!echo "##### In validation/dogs/ #####"
!ls validation/dogs/ |head

sample_data  train  train.tar  validation  validation.tar
##### In train/ #####
cats  dogs
##### In validation/ #####
cats  dogs
##### In train/cats/ #####
cat.1000.jpg
cat.1001.jpg
cat.1002.jpg
cat.1003.jpg
cat.1004.jpg
cat.1005.jpg
cat.1006.jpg
cat.1007.jpg
cat.1008.jpg
cat.1009.jpg
##### In train/dogs/ #####
dog.1000.jpg
dog.1001.jpg
dog.1002.jpg
dog.1003.jpg
dog.1004.jpg
dog.1005.jpg
dog.1006.jpg
dog.1007.jpg
dog.1008.jpg
dog.1009.jpg
##### In validation/cats/ #####
cat.100.jpg
cat.101.jpg
cat.102.jpg
cat.103.jpg
cat.104.jpg
cat.105.jpg
cat.106.jpg
cat.107.jpg
cat.108.jpg
cat.109.jpg
##### In validation/dogs/ #####
dog.100.jpg
dog.101.jpg
dog.102.jpg
dog.103.jpg
dog.104.jpg
dog.105.jpg
dog.106.jpg
dog.107.jpg
dog.108.jpg
dog.109.jpg


## Quiz Question 2
Please load the InceptionV3 model as the base model.

Hint: https://keras.io/applications/#fine-tune-inceptionv3-on-a-new-set-of-classes

In [ ]:
# Your codes here
############################################################

# in this example, we will use inception V3 as the base model.
# you will have the chance to try a number of ImageNet models in Assignment 2.
base_model = InceptionV3(weights='imagenet', include_top=False)


87910968/87910968 [==============================] - 5s 0us/step


## Quiz Question 3
Add a logistic layer for binary classification

Hint: https://keras.io/applications/#fine-tune-inceptionv3-on-a-new-set-of-classes

In [ ]:
# Your codes here
############################################################

# get the output of the base model
x = base_model.output
# add a 2D global average pooling layer
x = GlobalAveragePooling2D()(x)
# add a layer for binary classification
predictions = Dense(1, activation='sigmoid')(x)

# define the model to be trained
model = Model(inputs=base_model.input, outputs=predictions)


###To fix the layers in the base model.
It is useful if you don't have a lot of data. If you have sufficient data, you can also backpropagate into the layers in the base model after the last layers were trained.

In [ ]:
# only train the added layers, but not the layers in the base model
for layer in base_model.layers:
    layer.trainable = False


## Quiz Question 4
Compile the model.
Why do we need to compile the model?

Hint: https://keras.io/models/model/#compile

In [ ]:
# Your answer and codes here
############################################################
# in Keras, we need to compile the model to configure the optimizer, loss function, etc.
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])


###Define data pre-processing methods for the training and the test set

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# define the batch size, if there is sufficient GPU memory, you can increase the batch size
batch_size = 16

###Define the methods to load the image from directories
Please put your training data under the 'train' directory, with images with label "0" under 'train/0' and images with label "1" under 'train/1'; put validation data under the 'validation' directory, with images with label "0" under 'validation/0' and images with label "1" under 'validation/1'

In [ ]:
train_generator = train_datagen.flow_from_directory(
        'train',  # the directory for the training data
        target_size=(150, 150),  # resize the input images to accommodate the model
        batch_size=batch_size,
        class_mode='binary')


validation_generator = test_datagen.flow_from_directory(
        'validation', # the directory for the validation data
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')


Found 2000 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


### Train the model

In [ ]:
# please change nTraining to the number of training cases and nValidation to the number of validation cases
nTraining = 2000
nValidation = 200
model.fit(
        train_generator,
        steps_per_epoch=nTraining // batch_size,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=nValidation // batch_size)

Epoch 1/5
125/125 [==============================] - 32s 150ms/step - loss: 0.2137 - accuracy: 0.9130 - val_loss: 0.1143 - val_accuracy: 0.9479
Epoch 2/5
125/125 [==============================] - 16s 131ms/step - loss: 0.1299 - accuracy: 0.9490 - val_loss: 0.1358 - val_accuracy: 0.9375
Epoch 3/5
125/125 [==============================] - 18s 147ms/step - loss: 0.1050 - accuracy: 0.9590 - val_loss: 0.1366 - val_accuracy: 0.9479
Epoch 4/5
125/125 [==============================] - 17s 137ms/step - loss: 0.1081 - accuracy: 0.9535 - val_loss: 0.1402 - val_accuracy: 0.9427
Epoch 5/5
125/125 [==============================] - 17s 138ms/step - loss: 0.0959 - accuracy: 0.9640 - val_loss: 0.1317 - val_accuracy: 0.9583


## Quiz Question 5
How to accelerate the training process?

In [ ]:
# Your answer here
############################################################
# Runtime -> Change runtime type -> Hardware accelerator -> GPU

### Generate predictions on the validation set

In [ ]:
generator = test_datagen.flow_from_directory(
        'validation',
        target_size=(150, 150),
        batch_size=100,
        class_mode=None,
        shuffle=False)

probabilities = model.predict(generator, steps=2)

Found 200 images belonging to 2 classes.
2/2 [==============================] - 4s 20ms/step


### Generate the confusion matrix

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

y_true = np.array([0] * 100 + [1] * 100)
y_pred = probabilities > 0.5

confusion_matrix_inceptionV3 = confusion_matrix(y_true, y_pred)
print(confusion_matrix_inceptionV3)

[[92  8]
 [ 1 99]]


## Quiz Question 6
What's the validation accuracy based on y_pred?

In [ ]:
# Your codes here
############################################################
np.trace(confusion_matrix_inceptionV3)/np.sum(confusion_matrix_inceptionV3)

0.955